### Pedalcast: Predicting daily bike usage for each station - Forecasting Notebook

#### Imports

In [39]:
import numpy as np
import pandas as pd
import plotly.express as px

from collections import defaultdict
from itertools import groupby
from operator import itemgetter
import joblib
from functions.features import create_features

In [40]:
daily_counts = pd.read_csv('data/daily_counts_with_weather.csv')
daily_counts['date'] = pd.to_datetime(daily_counts['date'])
daily_counts.head()

,start_station_id,start_station_name,date,trip_count,temp,humidity,precip,snow,windspeed,visibility,cloudcover
0,A32000,Fan Pier,2023-07-01,30,19.3,93.5,0.000,0.0,20.3,4.8,90.6
1,A32000,Fan Pier,2023-07-02,20,20.9,89.9,8.278,0.0,28.9,7.6,98.5
2,A32000,Fan Pier,2023-07-03,28,20.4,89.4,14.249,0.0,18.0,10.4,89.5
3,A32000,Fan Pier,2023-07-04,35,20.1,94.4,12.163,0.0,16.4,12.0,96.0
4,A32000,Fan Pier,2023-07-05,41,23.9,78.4,0.000,0.0,17.8,13.3,53.3


#### Detecting Seasonal Inactivity

To avoid forecasting for stations during periods when they’ve never been active, we identify seasonal inactivity as follows:

1. Normalize across years: Add `dayofyear` and `year` to align activity by calendar day.
2. Build activity matrix: Mark each station-day as active if `trip_count > 0`.
3. Invert to inactivity: Identify days that were never active across any year.
4. Group into spans: Detect consecutive inactive days and retain blocks ≥ 45 days.
5. Store results: Save (start_day, end_day) tuples per station to filter future forecasts.

This avoids predictions during seasonal closures (e.g. winter) while still allowing forecasts for newer stations.

In [41]:
# 1. Create a 'day of year' column to normalize across years
daily_counts['dayofyear'] = daily_counts['date'].dt.dayofyear
daily_counts['year'] = daily_counts['date'].dt.year

# 2. Pivot: For each station and dayofyear, see if there was *any* activity
activity_matrix = (
    daily_counts.groupby(['start_station_id', 'year', 'dayofyear'])['trip_count']
    .sum()
    .reset_index()
)

# 3. Mark whether activity occurred (1) or not (0)
activity_matrix['active'] = activity_matrix['trip_count'] > 0

# 4. Aggregate across all available years: was a station ever active on this day?
ever_active = (
    activity_matrix
    .groupby(['start_station_id', 'dayofyear'])['active']
    .any()
    .unstack(fill_value=False)  # rows: stations, columns: 1–366
)

# 5. Find periods where the station was *never* active on any year
never_active = ~ever_active  # Flip to inactivity

# 6. Detect continuous inactivity spans (e.g., 11/14 to 3/14 ~ days 318–73)
seasonal_inactivity = defaultdict(list)

for station_id, inactive_days in never_active.iterrows():
    inactive_doys = inactive_days[inactive_days].index.tolist()

    if not inactive_doys:
        continue

    # Group consecutive dayofyear values

    for k, g in groupby(enumerate(inactive_doys), lambda x: x[1] - x[0]):
        block = list(map(itemgetter(1), g))
        if len(block) >= 45:  # Only consider "seasonal" if >45 days
            seasonal_inactivity[station_id].append((block[0], block[-1]))

# 7. Show some examples
for station, blocks in list(seasonal_inactivity.items())[:5]:
    print(f"{station}: inactive blocks → {blocks}")

A32014: inactive blocks → [(1, 155), (182, 366)]
A32021: inactive blocks → [(1, 118), (182, 366)]
A32024: inactive blocks → [(182, 275)]
A32029: inactive blocks → [(1, 72)]
A32034: inactive blocks → [(1, 127)]


#### Forecasting Future Bike Demand

To generate predictions for future dates, we define a `forecast_bike_demand()` function that simulates each day recursively.

- Historical data (including trip counts) is combined with a date-station grid for the prediction period.
- For each forecasted date, the function:
  1. Creates features including lag and rolling statistics using the latest available data
  2. Uses the trained XGRegressor model to predict demand for every station on that day
  3. Appends the predictions back into the dataset so they can be used as lags in the next day's forecast
- This recursive structure allows the model to forecast multiple consecutive days, even when no true data exists beyond the last historical date.
- The final result is a clean DataFrame of predicted trip counts per station for each day for the requested date range.

To support forecasting, we also provide the model with future weather data from July 1, 2025 to July 1, 2027, downloaded from the [Visual Crossing API](https://www.visualcrossing.com/). This ensures the model can incorporate temperature, precipitation, wind, and other conditions during prediction. You can also use it later to get more dates if needed.

##### Skipping Seasonally Inactive Stations

Some stations are consistently closed during specific parts of the year, such as winter. To avoid generating unrealistic predictions for these periods, we use an `is_inactive()` function:

- It checks whether a station is inactive on a given day-of-year based on historical patterns based on the `seasonal_inactivity` dict we defined in the cell before this.
- The forecast function skips predictions for (station, date) pairs where inactivity is detected.

#### Visualizing Predictions

We use the `plot_predictions()` function to create an interactive Plotly line chart of predicted demand over time, grouped by station.

- `top_n`(default = 10) Number of stations to show, based on total predicted trips during the forecast period. Only used if `station_ids` is not specified.
- `station_ids`(optional) A list of specific `start_station_id`s to visualize. If provided, this overrides `top_n`.

Below, we apply the forecasting function to predict demand from **July 1 to July 7, 2025** using the trained model and visualize the predictions.

In [42]:
# future weather forecast from July 1 2025 through July 1 2027
weather_future = pd.read_csv('data/weather_future.csv')[['datetime','temp','humidity','precip','snow','windspeed','visibility','cloudcover']]
weather_future['date'] = pd.to_datetime(pd.to_datetime(weather_future['datetime']).dt.date)
weather_future.tail()

,datetime,temp,humidity,precip,snow,windspeed,visibility,cloudcover,date
726,2027-06-27,22.5,68.6,3.1,NaN,25.9,12.5,46.2,2027-06-27
727,2027-06-28,22.5,71.3,4.2,NaN,25.6,13.4,64.6,2027-06-28
728,2027-06-29,22.5,71.3,3.1,NaN,21.6,13.9,58.0,2027-06-29
729,2027-06-30,23.1,65.1,1.3,NaN,23.8,14.3,52.9,2027-06-30
730,2027-07-01,22.9,68.5,4.4,NaN,25.2,13.6,53.2,2027-07-01


In [43]:
def forecast_bike_demand(start_date, end_date, full_data, model_pipeline, station_ids=None):
    """
    Forecasts bike demand per station from start_date to end_date using past data and a trained pipeline.
    """

    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)

    # 1. Filter to historical data needed for lags
    history_window = (full_data[['start_station_id','date','trip_count']].copy())
    
    # 2. Get list of all active stations(1 year) to forecast if user doesn't choose specific stations
    if station_ids is None:
        stations = (full_data[
    (full_data['date'] >= '2024-06-30') & (full_data['date'] <= '2025-06-30')]
    ['start_station_id'].unique())
    else:
        stations = station_ids
    
    recursive_start = pd.to_datetime("2025-07-01")

    # 3. Create empty grid of future dates × stations
    future_dates = pd.date_range(start=recursive_start, end=end_date, freq='D')
    future_df = pd.DataFrame([(station, date) for station in stations for date in future_dates],
                             columns=['start_station_id', 'date'])
    
    # 4. Combine with historical data (to allow lag feature generation)
    combined = pd.concat([history_window, future_df], ignore_index=True, sort=False)

    # 5. Create placeholder for predictions
    all_predictions = []

    # 6. Forecasting window
    forecast_dates = pd.date_range(
        start=recursive_start,
        end=end_date,
        freq='D'
    )

    # 7. Start with combined base data (historical + blank rows)
    combined = pd.concat([history_window, future_df], ignore_index=True, sort=False)
    combined = combined.merge(weather_future, on='date', how='left')

    for forecast_date in forecast_dates:
        # Create features with latest known + predicted values
        temp = create_features(combined.copy())

        # Filter to the row for this forecast date
        row = temp[temp['date'] == forecast_date]
        if seasonal_inactivity:
            row = row[~row.apply(
                lambda r: is_inactive(r['start_station_id'], r['date'], seasonal_inactivity),
                axis=1
            )]
        if row.empty:
            continue

        # Predict
        X_row = row.drop(columns=['trip_count'], errors='ignore')
        y_pred = model_pipeline.predict(X_row)

        row['trip_count'] = y_pred
        all_predictions.append(row[['date', 'start_station_id', 'trip_count']])

        # Append prediction back to combined to use in future lags
        combined = pd.concat([combined, row[['start_station_id', 'date', 'trip_count']]], ignore_index=True)

    # 8. Final result
    result_df = pd.concat(all_predictions, ignore_index=True)
    result_df.rename(columns={'trip_count': 'predicted_trip_count'}, inplace=True)
    result_df = result_df[(result_df['date'] >= start_date) & (result_df['date'] <= end_date)]

    return result_df[['date', 'start_station_id', 'predicted_trip_count']]

def is_inactive(station_id, date, inactivity_dict):
    doy = date.timetuple().tm_yday
    for start, end in inactivity_dict.get(station_id, []):
        if start <= doy <= end:
            return True
    return False

def plot_predictions(predictions_df, top_n=10,station_ids=None):
    if station_ids is not None:
        plot_df = predictions_df[predictions_df['start_station_id'].isin(station_ids)]
    else:
        top_stations = (
            predictions_df.groupby('start_station_id')['predicted_trip_count']
              .sum()
              .sort_values(ascending=False)
              .head(top_n)
              .index
        )
        plot_df = predictions_df[predictions_df['start_station_id'].isin(top_stations)]

    fig = px.line(
        plot_df,
        x="date",
        y="predicted_trip_count",
        color="start_station_id",
        title="Predicted Daily Bike Usage (Top Stations)",
        labels={"predicted_trip_count": "Predicted Trips"},
        template="plotly_white"
    )
    fig.show()

In [44]:
# Load saved model
loaded_model = joblib.load('xgb_final_model.pkl')

# Forecast demand for the first week of July 2025
predictions = forecast_bike_demand(
    start_date='2025-07-01',
    end_date='2025-07-07',
    full_data=daily_counts,  
    model_pipeline=loaded_model
)

predictions

,date,start_station_id,predicted_trip_count
0,2025-07-01,A32000,10.158089
1,2025-07-01,A32001,12.152802
2,2025-07-01,A32002,37.023739
3,2025-07-01,A32003,23.674147
4,2025-07-01,A32004,33.753563
...,...,...,...
3453,2025-07-07,W32009,8.406739
3454,2025-07-07,X32999,8.996682
3455,2025-07-07,Z32998,10.197105
3456,2025-07-07,Z32999,6.220559


In [45]:
plot_predictions(predictions)